In [1]:
import numpy as np
import math
import pprint as pp
from scipy import *
from scipy.sparse import *

## Generation de graphes

In [2]:
handmadeGraph1 = {1: {2:0.5,3:0.5},
                 2: {4:0.6},
                 3: {4:0.4,5:0.6},
                 4: {},
                 5: {},
                }

handmadeGraph=dok_matrix((7,7),dtype=float32)
handmadeGraph[0,1] =0.5
handmadeGraph[0,2] =0.5
handmadeGraph[1,3] =0.6
handmadeGraph[2,3] =0.4
handmadeGraph[2,4] =0.6

print("Handmade Graph")
print(handmadeGraph)


nodes = np.arange(handmadeGraph.shape[0])

def randomGraph(nodes):
    m=max(nodes)+1
    graph = dok_matrix((m,m),dtype=float32)
    for n1 in nodes:
        for n2 in nodes:
            if(n1!=n2):
                graph[n1,n2]=np.random.rand()
    return graph
print("Random Graph")
print(randomGraph(nodes))

Handmade Graph
  (0, 1)	0.5
  (0, 2)	0.5
  (1, 3)	0.6
  (2, 3)	0.4
  (2, 4)	0.6
Random Graph
  (0, 1)	0.6668038
  (0, 2)	0.22190805
  (0, 3)	0.544171
  (0, 4)	0.19102918
  (0, 5)	0.8954404
  (0, 6)	0.5133796
  (1, 0)	0.1304498
  (1, 2)	0.4974894
  (1, 3)	0.5976379
  (1, 4)	0.950712
  (1, 5)	0.32473055
  (1, 6)	0.74529326
  (2, 0)	0.217423
  (2, 1)	0.8162855
  (2, 3)	0.1129149
  (2, 4)	0.87297654
  (2, 5)	0.7347039
  (2, 6)	0.20040265
  (3, 0)	0.94846225
  (3, 1)	0.96577746
  (3, 2)	0.6390994
  (3, 4)	0.2123754
  (3, 5)	0.5311662
  (3, 6)	0.4648414
  (4, 0)	0.17772995
  (4, 1)	0.97153217
  (4, 2)	0.4636219
  (4, 3)	0.495635
  (4, 5)	0.6778511
  (4, 6)	0.99952847
  (5, 0)	0.66017884
  (5, 1)	0.12974547
  (5, 2)	0.0077006654
  (5, 3)	0.9104476
  (5, 4)	0.59879464
  (5, 6)	0.952536
  (6, 0)	0.88597757
  (6, 1)	0.3391013
  (6, 2)	0.643241
  (6, 3)	0.29102704
  (6, 4)	0.71193
  (6, 5)	0.53580844


## Inference 
Generation de cascades

In [3]:

def genCascade(graph,startNode,startTime=0):
    ''' Receive graph and starting infected node generate a cascade 
        Args
            graph(node to child representation)
            cascade {node : time} 
    '''
    cascade = {startNode : startTime}
    lastInfected = [startNode]
    infected_next = {}
    time = startTime+1
    while len(lastInfected)> 0:
        for infected in lastInfected:
            for (_,child),pct in graph[infected,:].items():
                if (child not in cascade) and (child not in infected_next):
                    if (np.random.rand()<pct):
                        infected_next[child] = time
        cascade.update(infected_next)
        lastInfected = list(infected_next.keys())
        infected_next = {}
        time +=1
        
    return cascade

In [4]:
nbCascades = 200
cascades = [genCascade(handmadeGraph,0) for i in range(nbCascades)]
pp.pprint(cascades[:3])

[{0: 0, 2: 1, 3: 2, 4: 2}, {0: 0, 1: 1, 3: 2}, {0: 0, 1: 1}]


## Independant cascades apprentissage


### 2008 Saito
Les temps d'infections contiguë <br>
$D(i)$  all newly infected at time $i$  <br>
$D = D(0) \cup D(1) \cup ... \cup D(n)$
#### Maximimum likelyhood expectation
Etapes : 
1. On donne une première estimation des arêtes du graph : $k_{u,v}$ <br>
- On calcule $P_{w}$ en fonction de $k_{u,v}$
- On calcule par ML $k_{u,v}$ en fonction de $P_{w}$
- Retour à l'étape 2 jusqu'à convergence


In [5]:
def cascade_repr(cascade):
    ''' Transforme une cascade en list ayant pour indice 
        le temps d'infection t et pour contenu une list contenant les noeuds infectés au temps t'''
    
    maxT  = max(cascade.values())+1
    Ds = [[] for i in range(maxT)]
    for (n,t) in cascade.items():
        Ds[t].append(n)
    return Ds
D = [cascade_repr(c) for c in cascades]
print(cascades[:2],'\n',D[:2])

[{0: 0, 2: 1, 3: 2, 4: 2}, {0: 0, 1: 1, 3: 2}] 
 [[[0], [2], [3, 4]], [[0], [1], [3]]]


In [6]:
def nodes_in_Ds(Ds):
    uniques = []
    for nodes in Ds : 
        for n in nodes : 
            uniques.append(n)
    return uniques

In [7]:

def P_sw(g,Ds,w):
    ''' Vraisemblance de l infection d un noeud sachant le graphe 
    et les noeuds acivés à l'etape precedente '''
    not_activated = 1
    t = None
    for i,nodes in enumerate(Ds):
        if (w in nodes):
            t = i
    if (t == 0): # si le noeud est le premier
        return 1
    if (t is None): # si le noeud n'est pas dans l'episode de diffusion
        raise Exception(f"node {w} is not in diff episode : {Ds} ")
    else :  # si le noeud est dans l'épisode de diffusion
        return 1 - np.prod ([1-g[parent,w] for parent in Ds[t-1]])

In [8]:
def Expectation(g,D):
    ''' Calcule l'ensemble des P_ws 
        P_ws[idD][node]->proba'''
    p_sw = [{n:P_sw(g,Ds,n) for n in nodes_in_Ds(Ds)} for Ds in D]
    return p_sw

In [9]:
def D_plus_uv_id(D,u,v):
    ''' pour chaque couples u,v renvoit l'ensemble des 
    Ds (episode infections) ou u precede v'''
    D_plus = []
    for i,Ds in enumerate(D): 
        for t in range(1,len(Ds)):
            if (u in Ds[t-1] and v in Ds[t]):
                D_plus.append(i)
                break
    return D_plus

def D_minus_uv_len(D,u,v):
    '''Pour chaque couple u,v renvoit le cardinal de l'ensemble des 
    Ds(episode infection) ou u est present mais v ne le suit pas'''
    D_minus_len=0
    for Ds in D : 
        for t in range(1,len(Ds)):
            if (u in Ds[t-1] and v not in Ds[t]):
                D_minus_len+=1
                break
        if (u in Ds[-1]):
            D_minus_len+=1
    return D_minus_len

In [10]:
def Maximisation_uv(g,D_plus_id,Dminus_len,p_sw,u,v):
    '''Calcule les nouveaux paramètre pour l'arete u,v '''
    if ((len(D_plus_id[u][v])+Dminus_len[u][v]) == 0):
        #raise Exception(f"{u}-{v} Division zero")
        return 0
    return (1/(len(D_plus_id[u][v])+Dminus_len[u][v])) *sum([g[u,v]/p_sw[i][v] for i in D_plus_id[u][v]])


def Maximisation(g,D_plus_id,Dminus_len,p_sw):
    ''' Calcule les nouveaux paramètres pour le graphe'''
    gprime = dok_matrix(g.shape,dtype=float32)
    for u,v in g.keys():
        if u != v:
            gprime[u,v] = Maximisation_uv(g,D_plus_id,Dminus_len,p_sw,u,v)
    return gprime


In [11]:
def EM_IC(D,nodes):
        # initalisation
    g = randomGraph(nodes)    
    p_sw = None
    D_plus_id =   {v:{u:D_plus_uv_id(D,v,u) for u in nodes} for v in nodes}
    D_minus_len = {v:{u:D_minus_uv_len(D,v,u)for u in nodes} for v in nodes}
    for i in range(100):
        p_sw = Expectation(g,D)
        g = Maximisation(g,D_plus_id,D_minus_len,p_sw)
    return g

nodes = set()
for c in cascades:
    nodes.update(c.keys())
print(nodes)

D = [cascade_repr(c) for c in cascades]
finalGraph = EM_IC(D,nodes)
print(finalGraph)

{0, 1, 2, 3, 4}
  (0, 1)	0.5
  (0, 2)	0.51
  (1, 3)	0.58487135
  (1, 4)	4.70519e-32
  (2, 3)	0.43633896
  (2, 4)	0.54901963


### Methode 2
Les temps d'infections non contiguë : 
Un noeud peut désorais être infecté non seulement par ceux qui ont été infecté

In [12]:
def P_sw2(g,Ds,w):
    ''' Vraisemblance de l infection d un noeud sachant le graphe 
    et les tous noeuds précedemment acivés '''
    not_activated = 1
    t = None
    for i,nodes in enumerate(Ds):
        if (w in nodes):
            t = i
    if (t == 0): # si le noeud est le premier
        return 1
    if (t is None): # si le noeud n'est pas dans l'episode de diffusion
        raise Exception(f"node {w} is not in diff episode : {Ds} ")
    else :  # si le noeud est dans l'épisode de diffusion
        preceding_nodes = [] # on regroupe les noeuds des épisodes précedents
        for nodes in Ds[:t]:
            for n in nodes : 
                preceding_nodes.append(n)
        
        return 1 - np.prod ([1-g[parent,w] for parent in preceding_nodes])

In [13]:
def Expectation2(g,D):
    ''' Calcule l'ensemble des P_ws 
        P_ws[idD][node]->proba'''
    p_sw = [{n:P_sw2(g,Ds,n) for n in nodes_in_Ds(Ds)} for Ds in D]
    return p_sw

In [14]:
def D_plus_uv_id2(D,u,v):
    ''' Pour chaque couples u,v renvoit l'ensemble des 
        Ds (episode infections) ou t_u < t_v'''
    D_plus = []
    
    for i,Ds in enumerate(D) : 
        preceding_nodes = []
        for t in range(0,len(Ds)):
            if (v in Ds[t]):
                if (u in preceding_nodes):
                    D_plus.append(i)
                    break
            else : 
                preceding_nodes +=Ds[t]
    return D_plus

def D_minus_uv2_len(D,u,v):
    '''Pour chaque couple u,v renvoit l'ensemble des 
    Ds(episode infection) ou u est present et non(t_u < t_v)'''
    D_minus_len = 0
    for Ds in D : 
        u_in_Ds = False
        v_in_Ds = False
        for t in range(0,len(Ds)):
            if (u in Ds[t]):
                u_in_Ds = True
            if (u_in_Ds and v in Ds[t]):
                v_in_Ds = True
                break
        if (u_in_Ds and not v_in_Ds):
            D_minus_len+=1
    return D_minus_len

In [15]:
def EM_IC_2(D,nodes):
    g = randomGraph(nodes)    
    p_sw = None
    D_plus_id =   {v:{u:D_plus_uv_id2(D,v,u) for u in nodes} for v in nodes}
    D_minus_len = {v:{u:D_minus_uv2_len(D,v,u)for u in nodes} for v in nodes}
    
    for i in range(100):
        p_sw = Expectation2(g,D)
        g = Maximisation(g,D_plus_id,D_minus_len,p_sw)
    return g

D = [cascade_repr(c) for c in cascades]
finalGraph = EM_IC_2(D,nodes)
print(finalGraph)

  (0, 1)	0.5
  (0, 2)	0.51
  (0, 3)	4.7499366e-11
  (0, 4)	1.5377493e-30
  (1, 3)	0.58487135
  (1, 4)	5.263753e-32
  (2, 3)	0.43633896
  (2, 4)	0.54901963


## Evaluation

On va évaluer les modèles par MeanAveragePrecision (MAP): <br>
On peut caculer $AP$ pour un épisode ($Ds$) en particulier : <br><br>
$ AP(Ds) =  \sum_{i=1}^{|U^{Ds}|} \frac{ |\{ U_1^{Ds},...,U_i^{Ds} \} \cap Ds |}{i}  dx$ <br>
$ AP(Ds) = \sum_{i=1}^{|U^{Ds}|} \frac{TruePositive}{TruePositive+FalseNegative}dx$ <br><br>
$ dx = recall(i) -recall(i-1) $<br>
<br>
$ MAP = \frac{1}{D}\sum_{Ds\in |D|} AP(Ds)$

On va d'abord calculer les probabilité d'infection d'un noeud connaissant la source 
par génération d'épisodes : 

In [16]:
def Pws_gs(graph,source,nbEpisode=100):
    ''' Calcule la probabilité qu'un noeud soit infecté connaissant 
        une source et un graph de diffusion
        On calcule par moyenne sur echantionnage d'episodes d'infections '''
    proba_infected = {n:0 for n in range(graph.shape[0])}
    for i in range(nbEpisode):
        c = genCascade(graph,source)
        for node in c : 
            proba_infected[node]+=1/nbEpisode
    return proba_infected

In [17]:
def AP(Ds,graph):
    '''Average Precision pour un episode Ds '''
    pws_gs =Pws_gs(graph,Ds[0][0])
    U_d = sorted(pws_gs,key=pws_gs.get,reverse=True) # sort par ordre decroissant
    ap = 0
    DsNodeSet = set(nodes_in_Ds(Ds)) # noeuds faisant partie de l'episode d'infection Ds
    
        # calcul des points de precision et de recall : 
    precision = np.ones((len(U_d)+1))
    recall = np.zeros((len(U_d)+1))
    for i in range(1,len(precision)):
        tp = len(DsNodeSet.intersection(U_d[:i])) # TruePositive
        precision[i] = tp/i
        recall[i] = tp/len(DsNodeSet)
        
        # smooth precision curve
    for i in range(len(precision)-2,-1,-1):
        precision[i] = max(precision[i],precision[i+1])
    
    ap = 0
    for i in range(1,len(precision)):
        dx = recall[i]-recall[i-1]
        ap+= precision[i]* dx

    return ap

def MAP(D,graph):
    return sum([AP(Ds,graph)for Ds in D])/len(D)    

In [18]:
print("original  graph MAP score : ",MAP(D,handmadeGraph))
print("inference graph MAP score : ",MAP(D,finalGraph))

original  graph MAP score :  0.9368333333333334
inference graph MAP score :  0.9268611111111111


# Test avec matrices Sparses : 

In [19]:
def GenerateData(nbNodes,nbCascades,density=0.1):
    ''' Genere un graph aléatoire et des données '''
    sparseGraph = dok_matrix(random(nbNodes,nbNodes,density=density))
    cascades = []
    for i in range(nbCascades):
        startNode = np.random.randint(nbNodes)
        cascades.append(genCascade(sparseGraph,startNode))
        # cascade representation : 
    D = [cascade_repr(c) for c in cascades]
    return sparseGraph,D

def ModelScore(D,trainD,testD):
    ''' Apprends un modele sur ensemble de trainD 
        et renvoi le résultat sur un ensemble de testD'''
    nodes = set()
    for Ds in D:
        for Ds_t in Ds : 
            nodes.update(set(Ds_t))
    predictedGraph = EM_IC_2(trainD,nodes)
    return MAP(testD,predictedGraph)

In [21]:
# Params : 
nbNodes = 200
nbCascades = 400
originalGraph,D = GenerateData(nbNodes,nbCascades,0.007)
trainD,testD = D[int(nbCascades/2):],D[:int(nbCascades/2)]
predictedScore = ModelScore(D,trainD,testD)
originalScore = MAP(D,originalGraph)
print("original  graph MAP score : ",originalScore)
print("predictedGraph MAP score : ",predictedScore)

original  graph MAP score :  0.9600711480474993
predictedGraph MAP score :  0.8040477578916287
